In [8]:
# 라이브러리 및 모델 불러오기

from flask import Flask ,render_template
from flask import request, redirect
from konlpy.tag import Kkma,Okt, Twitter, Komoran # 형태소 분석 라이브러리
from moviepy.editor import * # 영상을 오디오 파일로 변환
from moviepy.editor import VideoFileClip, concatenate_videoclips
import moviepy.editor as mp
from pytube import YouTube # 유튜브 영상 다운로드 또는 불러오기
import pytube
import tqdm as tq

import kss # 텍스트 문장으로 바꾸는 라이브러리
import speech_recognition as sr # 오디오 파일 또는 음성을 텍스트로 변환
import pandas as pd
# BOW = BAG of WORD : 단어가방, 단어모음, 단어사전
from sklearn.feature_extraction.text import CountVectorizer
# 위 도구는 빈도수 기반 벡터화 도구

# # 알고리즘 시작

# # 형태소 구분하는 함수
# - 사용해야할 품사가 생각보다 많음 ( komoran 기준 ) 
#   - 명사 NN -> 일반명사 NNG // 고유명사 NNP // 의존명사 NNB

# 형태소 구분 함수
def lemmatize(word):
    morphtags = Komoran().pos(word)
    if morphtags[0][1] == 'NNG' or morphtags[0][1] == 'NNP':
        return morphtags[0][0]

In [3]:

# +
# https://www.youtube.com/watch?v=kFnHWpGs-18  :: 스마트 인재 개발원
# https://www.youtube.com/watch?v=lZi3k_GzfCk  :: 서강대 2:25 ~ 4: 10
youtube=input('다운로드 받을 유튜브 영상 링크 : ')

yt = pytube.YouTube(youtube)

title = yt.title
# -


다운로드 받을 유튜브 영상 링크 : https://www.youtube.com/watch?v=xxv05lAduhk


In [4]:


# 영상 다운로드 경로

stream = yt.streams.all()[0]
stream.download(output_path='C:/Users/smhrd/Desktop/Machine Learning/test/data') 

# 영상 오디오 파일로 변환 
clip = mp.VideoFileClip("data/python 01.3gpp")
newsound = clip.subclip("00:01:10","00:01:30") # 20 sec
newsound.audio.write_audiofile("data/audio5.wav",16000,2,2000,'pcm_s16le')

# 오디오 파일 로드
filename = "data/audio5.wav"

# 오디오 파일 텍스트 추출
text = []
r = sr.Recognizer()
with sr.AudioFile(filename) as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data,language='ko-KR')
    # print(text)

# kss 활용 텍스트 문장 화
word_list = kss.split_sentences(text)

# 명사만 가져오기 위한 삭제
okt = Okt()
headline = []
stopwords = [ '의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','등','으로도']
for sentence in word_list:
    temp = []
    # morphs() : 형태소 단위로 토큰화
    # stem = True : 형태소에서 어간을 추출
    temp = okt.morphs(sentence, stem = True)
    temp = [word for word in temp if not word in stopwords]
    headline.append(temp)
    

# konlpy 트위터 이용 형태소 분류
twitter = Twitter()
sentences_tag = []
for word in headline:
    for i in word :
        morph = twitter.pos(i)
        sentences_tag.append(morph)
# print(sentences_tag)

# -

#  형태소 분류
noun_adj_list=[]
for i1 in sentences_tag:
    for word, tag in i1:
        if tag in ['Noun','Verb','Number','Adjective','Adverb']:
            noun_adj_list.append(word)
# print(noun_adj_list)

# 형태소 분류
for i in range(len(noun_adj_list)):
    #print(lemmatize(noun_adj_list[i]))
    if lemmatize(noun_adj_list[i]) != None :
        noun_adj_list[i] = lemmatize(noun_adj_list[i])
        print(noun_adj_list)

arr_list = noun_adj_list
print(arr_list)

 # 영상합치기 부분으로 넘어가기

[Korean Sentence Splitter]: <ipython-input-4-1934e5bd67b2>:1: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  stream = yt.streams.all()[0]

                                                                  

MoviePy - Writing audio in data/audio5.wav
MoviePy - Done.


[Korean Sentence Splitter]: Initializing Pynori...


보고 강의 목표에 대해 이야기 하겠습니다 그리고 다음 수업부터 본격적으로 파이썬 언어에 대한 학습을 시작할 수 있도록 파이썬언어를 설치하는 과정을 보겠습니다 교육이 점점 중요해지는 이유는 소프트웨어가 미래사회에서 요구하는 용량이고 속초의 발전으로


[Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')



['보고 강의 목표에 대해 이야기 하겠습니다', '그리고 다음 수업부터 본격적으로 파이썬 언어에 대한 학습을 시작할 수 있도록 파이썬언어를 설치하는 과정을 보겠습니다', '교육이 점점 중요해지는 이유는 소프트웨어가 미래사회에서 요구하는 용량이고 속초의 발전으로']
------------------------------------------------------------
[[('보고', 'Noun'), ('강의', 'Noun'), ('목표', 'Noun'), ('에', 'Josa'), ('대해', 'Noun'), ('이야기', 'Noun'), ('하겠습니다', 'Verb')], [('그리고', 'Conjunction'), ('다음', 'Noun'), ('수업', 'Noun'), ('부터', 'Josa'), ('본격', 'Noun'), ('적', 'Suffix'), ('으로', 'Josa'), ('파이썬', 'Noun'), ('언어', 'Noun'), ('에', 'Josa'), ('대한', 'Noun'), ('학습', 'Noun'), ('을', 'Josa'), ('시작', 'Noun'), ('할', 'Verb'), ('수', 'Noun'), ('있도록', 'Adjective'), ('파이썬', 'Noun'), ('언어', 'Noun'), ('를', 'Josa'), ('설치', 'Noun'), ('하는', 'Verb'), ('과정', 'Noun'), ('을', 'Josa'), ('보겠습니다', 'Verb')], [('교육', 'Noun'), ('이', 'Josa'), ('점점', 'Noun'), ('중요해지는', 'Adjective'), ('이유', 'Noun'), ('는', 'Josa'), ('소프트웨어', 'Noun'), ('가', 'Josa'), ('미래', 'Noun'), ('사회', 'Noun'), ('에서', 'Josa'), ('요구', 'Noun'), ('하는', 'Verb'), ('용량', 'Noun'), ('이고', 'Josa'), ('속초', '

In [93]:
# 영상 합치기
    
clips = []
for i in range(len(arr_list)):
    # print("*****", clips)
    try:
        # arr_list = ['보고', '강의', '목표', '대해', '이야기', '하다', '다음', '수업', '부터', '본격', '적', '파이썬', '언어', '대한', '학습', '시작', '하다', '수', '있다', '파이썬', '언어', '설치', '하다', '과정', '보다', '교육', '점점', '중요하다', '이유', '소프트웨어', '미래', '사회', '요구', '하다', '용량', '속초', '발전']
        mov = VideoFileClip("Data_Deep/30개영상/"+arr_list[i]+".mp4")
        clips.append(mov)
        print("try")
    except:
        print('skip')
        
    print("last", clips)
final_clip = concatenate_videoclips(clips, method='compose') # concat함수를 이용해 비디오를 합치기
final_clip.write_videofile("Success/sua5.mp4")

[Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\lib\site-packages\ipykernel\ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)



try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>]
try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>]
try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>]
try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138A00>]
try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFil

try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138A00>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138040>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138DF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138850>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138F10>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D69AE50>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B760>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11BFD0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B3A0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A1

try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138A00>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138040>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138DF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138850>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138F10>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D69AE50>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B760>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11BFD0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B3A0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A1

try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138A00>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138040>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138DF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138850>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138F10>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D69AE50>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B760>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11BFD0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B3A0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A1

try
last [<moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D66EAF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D68D9D0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138280>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138A00>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138040>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138DF0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138850>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D138F10>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D69AE50>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B760>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11BFD0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A10D11B3A0>, <moviepy.video.io.VideoFileClip.VideoFileClip object at 0x000001A1

t:   0%|          | 4/4350 [00:00<01:49, 39.86it/s, now=None]

Moviepy - Building video Success/sua5.mp4.
Moviepy - Writing video Success/sua5.mp4



t:   3%|▎         | 144/4350 [00:03<01:37, 43.32it/s, now=None][Korean Sentence Splitter]: C:\Users\smhrd\anaconda3\lib\site-packages\moviepy\video\io\ffmpeg_reader.py:123: UserWarning: Warning: in file Data_Deep/30개영상/보고.mp4, 6220800 bytes wanted but 0 bytes read,at frame 146/147, at time 4.87/4.87 sec. Using the last valid frame instead.
  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Done !
Moviepy - video ready Success/sua5.mp4


In [20]:
# 테스트

# 영상 다운로드 경로

stream = yt.streams.all()[0]
stream.download(output_path='C:/Users/smhrd/Desktop/Machine Learning/test/data') 

# 영상 오디오 파일로 변환 
clip = mp.VideoFileClip("data/python 01.3gpp")
newsound = clip.subclip("00:01:10","00:01:30") # 20 sec
newsound.audio.write_audiofile("data/audio5.wav",16000,2,2000,'pcm_s16le')

# 오디오 파일 로드
filename = "data/audio5.wav"

# 오디오 파일 텍스트 추출
text = []
r = sr.Recognizer()
with sr.AudioFile(filename) as source:
    audio_data = r.record(source)
    text = r.recognize_google(audio_data,language='ko-KR')
    # print(text)

# kss 활용 텍스트 문장 화
word_list = kss.split_sentences(text)

[Korean Sentence Splitter]: <ipython-input-20-3af1af8456a9>:3: DeprecationWarning: Call to deprecated function all (This object can be treated as a list, all() is useless).
  stream = yt.streams.all()[0]

                                                                  

MoviePy - Writing audio in data/audio5.wav
MoviePy - Done.
